# Data Analysis with Pandas

### Authors

###### Noemi Benci
Bachelor's Degree in Statistics.

###### Federico Pirona 
Bachelor's Degree in Statistics.

## Introduction
This analysis focuses on the spread of SARS-CoV-2 in Italy. 
Data are dowloded from these repositories on GitHub
- https://github.com/pcm-dpc/COVID-19/blob/master/dati-json/dpc-covid19-ita-regioni.json (Data by regions)
- https://github.com/pcm-dpc/COVID-19/blob/master/dati-json/dpc-covid19-ita-province.json (Data by province)

The data are updated every day from February, 24 2020 to June, 17, 2020 by the Depatment of the Civil Protecition. 
Data regards some day-by-day counts about the amount of infected, of deaths, of recovered and other conditions.
It has a natural tree structure because for each region there are one or more provinces and each pair is recorded daily. 

The analysis starts visualizing variables and the structures of the two datasets. 
Then we show some useful graphical representation of the data. After that we provide some data manipulation to get data more handle. 
Finally we combine together the two datasets and use a hierarchical indexing to make the dataset more readable.

## Loading data

In [4]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import date, datetime
import seaborn as sns
sns.set()

In [5]:
with open('G:/Algoritmi e programmazione/Progetto/Dati/dpc-covid19-ita-province.json') as f:
    d1 = json.load(f)

In [6]:
with open('G:/Algoritmi e programmazione/Progetto/Dati/dpc-covid19-ita-regioni.json') as f:
    d2 = json.load(f)

### Visualize data as dictionary

In [7]:
d1[:2]

[{'codice_provincia': 69,
  'codice_regione': 13,
  'data': '2020-02-24T18:00:00',
  'denominazione_provincia': 'Chieti',
  'denominazione_regione': 'Abruzzo',
  'lat': 42.35103167,
  'long': 14.16754574,
  'note_en': '',
  'note_it': '',
  'sigla_provincia': 'CH',
  'stato': 'ITA',
  'totale_casi': 0},
 {'codice_provincia': 66,
  'codice_regione': 13,
  'data': '2020-02-24T18:00:00',
  'denominazione_provincia': "L'Aquila",
  'denominazione_regione': 'Abruzzo',
  'lat': 42.35122196,
  'long': 13.39843823,
  'note_en': '',
  'note_it': '',
  'sigla_provincia': 'AQ',
  'stato': 'ITA',
  'totale_casi': 0}]

In [8]:
d2[:2]

[{'casi_testati': None,
  'codice_regione': 13,
  'data': '2020-02-24T18:00:00',
  'deceduti': 0,
  'denominazione_regione': 'Abruzzo',
  'dimessi_guariti': 0,
  'isolamento_domiciliare': 0,
  'lat': 42.35122196,
  'long': 13.39843823,
  'note_en': '',
  'note_it': '',
  'nuovi_positivi': 0,
  'ricoverati_con_sintomi': 0,
  'stato': 'ITA',
  'tamponi': 5,
  'terapia_intensiva': 0,
  'totale_casi': 0,
  'totale_ospedalizzati': 0,
  'totale_positivi': 0,
  'variazione_totale_positivi': 0},
 {'casi_testati': None,
  'codice_regione': 17,
  'data': '2020-02-24T18:00:00',
  'deceduti': 0,
  'denominazione_regione': 'Basilicata',
  'dimessi_guariti': 0,
  'isolamento_domiciliare': 0,
  'lat': 40.63947052,
  'long': 15.80514834,
  'note_en': '',
  'note_it': '',
  'nuovi_positivi': 0,
  'ricoverati_con_sintomi': 0,
  'stato': 'ITA',
  'tamponi': 0,
  'terapia_intensiva': 0,
  'totale_casi': 0,
  'totale_ospedalizzati': 0,
  'totale_positivi': 0,
  'variazione_totale_positivi': 0}]

### Visualize data as Pandas DataFrames

In [9]:
province = pd.DataFrame(d1)
province.sample(10)

,codice_provincia,codice_regione,data,denominazione_provincia,denominazione_regione,lat,long,note_en,note_it,sigla_provincia,stato,totale_casi
4425,4,1,2020-03-29T17:00:00,Cuneo,Piemonte,44.393296,7.551172,,,CN,ITA,591
6416,64,15,2020-04-14T17:00:00,Avellino,Campania,40.914047,14.795288,,,AV,ITA,402
8119,15,3,2020-04-27T17:00:00,Milano,Lombardia,45.466794,9.190347,,,MI,ITA,18559
7955,63,15,2020-04-26T17:00:00,Napoli,Campania,40.839566,14.250850,,,NA,ITA,2375
1251,89,19,2020-03-04T17:00:00,Siracusa,Sicilia,37.059917,15.293332,,,SR,ITA,0
5380,979,13,2020-04-06T17:00:00,In fase di definizione/aggiornamento,Abruzzo,0.000000,0.000000,,,,ITA,0
7811,67,13,2020-04-25T17:00:00,Teramo,Abruzzo,42.658918,13.704400,,,TE,ITA,633
6105,90,20,2020-04-11T17:00:00,Sassari,Sardegna,40.726677,8.559667,,,SS,ITA,724
4153,18,3,2020-03-27T17:00:00,Pavia,Lombardia,45.185093,9.160157,,,PV,ITA,1712
13101,11,7,2020-06-05T17:00:00,La Spezia,Liguria,44.107050,9.828190,,,SP,ITA,895


In [10]:
region = pd.DataFrame(d2)
region.sample(10)

,casi_testati,codice_regione,data,deceduti,denominazione_regione,dimessi_guariti,isolamento_domiciliare,lat,long,note_en,note_it,nuovi_positivi,ricoverati_con_sintomi,stato,tamponi,terapia_intensiva,totale_casi,totale_ospedalizzati,totale_positivi,variazione_totale_positivi
896,NaN,20,2020-04-06T17:00:00,47,Sardegna,56,670,39.215312,9.110616,,,15,123,ITA,7521,26,922,149,819,4
1222,38679.0,15,2020-04-22T17:00:00,327,Campania,860,2422,40.839566,14.250850,,,50,523,ITA,55701,53,4185,576,2998,52
275,NaN,21,2020-03-08T18:00:00,0,P.A. Bolzano,0,1,46.499335,11.356624,,,0,8,ITA,36,0,9,8,9,0
1493,20561.0,21,2020-05-05T17:00:00,286,P.A. Bolzano,1644,502,46.499335,11.356624,,,1,99,ITA,45264,11,2542,110,612,-24
2433,59912.0,10,2020-06-18T17:00:00,77,Umbria,1344,9,43.106758,12.388247,,,0,5,ITA,85586,2,1437,7,16,-2
174,NaN,6,2020-03-03T18:00:00,0,Friuli Venezia Giulia,0,12,45.649435,13.768136,,,4,1,ITA,354,0,13,1,13,4
1253,16094.0,20,2020-04-23T17:00:00,98,Sardegna,339,707,39.215312,9.110616,,,7,90,ITA,17844,20,1254,110,817,-16
1052,NaN,21,2020-04-14T17:00:00,214,P.A. Bolzano,406,1348,46.499335,11.356624,,,35,177,ITA,24457,39,2184,216,1564,27
1088,NaN,22,2020-04-15T17:00:00,318,P.A. Trento,798,1744,46.068935,11.121231,,,79,312,ITA,19535,48,3220,360,2104,22
2265,55763.0,10,2020-06-10T17:00:00,76,Umbria,1332,13,43.106758,12.388247,,,1,13,ITA,78366,2,1436,15,28,-1


## First look to 'Province' Dataset
Let's discover some info about the available variables and their nature.

In [11]:
province.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15104 entries, 0 to 15103
Data columns (total 12 columns):
codice_provincia           15104 non-null int64
codice_regione             15104 non-null int64
data                       15104 non-null object
denominazione_provincia    15104 non-null object
denominazione_regione      15104 non-null object
lat                        15104 non-null float64
long                       15104 non-null float64
note_en                    15104 non-null object
note_it                    15104 non-null object
sigla_provincia            15104 non-null object
stato                      15104 non-null object
totale_casi                15104 non-null int64
dtypes: float64(2), int64(3), object(7)
memory usage: 1.4+ MB


In [12]:
province.columns

Index(['codice_provincia', 'codice_regione', 'data', 'denominazione_provincia',
       'denominazione_regione', 'lat', 'long', 'note_en', 'note_it',
       'sigla_provincia', 'stato', 'totale_casi'],
      dtype='object')

In [13]:
province.describe(include='all')

,codice_provincia,codice_regione,data,denominazione_provincia,denominazione_regione,lat,long,note_en,note_it,sigla_provincia,stato,totale_casi
count,15104.000000,15104.000000,15104,15104,15104,15104.000000,15104.000000,15104,15104,15104,15104,15104.000000
unique,NaN,NaN,118,108,21,NaN,NaN,6,6,108,1,NaN
top,NaN,NaN,2020-03-12T17:00:00,In fase di definizione/aggiornamento,Lombardia,NaN,NaN,,,,ITA,NaN
freq,NaN,NaN,128,2478,1534,NaN,NaN,15099,15099,2478,15104,NaN
mean,207.523438,10.687500,NaN,NaN,NaN,35.868843,10.112000,NaN,NaN,NaN,NaN,1176.851298
std,347.394528,6.127435,NaN,NaN,NaN,16.072056,5.097761,NaN,NaN,NaN,NaN,2433.513404
min,1.000000,1.000000,NaN,NaN,NaN,0.000000,0.000000,NaN,NaN,NaN,NaN,0.000000
25%,32.750000,5.750000,NaN,NaN,NaN,38.848542,8.583304,NaN,NaN,NaN,NaN,38.000000
50%,64.500000,10.000000,NaN,NaN,NaN,42.609815,11.188560,NaN,NaN,NaN,NaN,348.000000
75%,96.250000,16.000000,NaN,NaN,NaN,44.947714,13.593962,NaN,NaN,NaN,NaN,1152.000000


The dataset of provinces is composed by 14 720 observations each composed by 12 variables. Each observation is a daily update of the count of cases of Coronavirus for a particular italian city and it provides all the information about the city (geographical location with longitude and latitude, which region does the city belong to) and the number of cases registred on a particular date. 

The dataset contains information of 115 different days for every region (21 unique regions) and province (107 provinces + 1 'In fase di aggiornamento') of Italy. 

On average have been registred 1159 cases per day, with a minimum of 0 and a maximum of about 24 000. 

##### Note that the latitude and the longitude reach as minumum 0. The rows with latitude and/or longitude have to be removed because wrong (each italian city has latitude and longitude different form 0).

In [14]:
province.shape

(15104, 12)

In [15]:
# identifying fake rows because of lat=0
province[province['lat']==0]

,codice_provincia,codice_regione,data,denominazione_provincia,denominazione_regione,lat,long,note_en,note_it,sigla_provincia,stato,totale_casi
4,979,13,2020-02-24T18:00:00,In fase di definizione/aggiornamento,Abruzzo,0.0,0.0,,,,ITA,0
7,980,17,2020-02-24T18:00:00,In fase di definizione/aggiornamento,Basilicata,0.0,0.0,,,,ITA,0
9,981,21,2020-02-24T18:00:00,In fase di definizione/aggiornamento,P.A. Bolzano,0.0,0.0,,,,ITA,0
15,982,18,2020-02-24T18:00:00,In fase di definizione/aggiornamento,Calabria,0.0,0.0,,,,ITA,0
21,983,15,2020-02-24T18:00:00,In fase di definizione/aggiornamento,Campania,0.0,0.0,,,,ITA,0
31,984,8,2020-02-24T18:00:00,In fase di definizione/aggiornamento,Emilia-Romagna,0.0,0.0,,,,ITA,0
36,985,6,2020-02-24T18:00:00,In fase di definizione/aggiornamento,Friuli Venezia Giulia,0.0,0.0,,,,ITA,0
42,986,12,2020-02-24T18:00:00,In fase di definizione/aggiornamento,Lazio,0.0,0.0,,,,ITA,0
47,987,7,2020-02-24T18:00:00,In fase di definizione/aggiornamento,Liguria,0.0,0.0,,,,ITA,0
60,988,3,2020-02-24T18:00:00,In fase di definizione/aggiornamento,Lombardia,0.0,0.0,,,,ITA,0


In [16]:
# identifying fake rows because of long=0
province[province['long']==0]

,codice_provincia,codice_regione,data,denominazione_provincia,denominazione_regione,lat,long,note_en,note_it,sigla_provincia,stato,totale_casi
4,979,13,2020-02-24T18:00:00,In fase di definizione/aggiornamento,Abruzzo,0.0,0.0,,,,ITA,0
7,980,17,2020-02-24T18:00:00,In fase di definizione/aggiornamento,Basilicata,0.0,0.0,,,,ITA,0
9,981,21,2020-02-24T18:00:00,In fase di definizione/aggiornamento,P.A. Bolzano,0.0,0.0,,,,ITA,0
15,982,18,2020-02-24T18:00:00,In fase di definizione/aggiornamento,Calabria,0.0,0.0,,,,ITA,0
21,983,15,2020-02-24T18:00:00,In fase di definizione/aggiornamento,Campania,0.0,0.0,,,,ITA,0
31,984,8,2020-02-24T18:00:00,In fase di definizione/aggiornamento,Emilia-Romagna,0.0,0.0,,,,ITA,0
36,985,6,2020-02-24T18:00:00,In fase di definizione/aggiornamento,Friuli Venezia Giulia,0.0,0.0,,,,ITA,0
42,986,12,2020-02-24T18:00:00,In fase di definizione/aggiornamento,Lazio,0.0,0.0,,,,ITA,0
47,987,7,2020-02-24T18:00:00,In fase di definizione/aggiornamento,Liguria,0.0,0.0,,,,ITA,0
60,988,3,2020-02-24T18:00:00,In fase di definizione/aggiornamento,Lombardia,0.0,0.0,,,,ITA,0


In [17]:
# before there were 14720 observation of which 2415 were fake. dropping the fake one we sould have 
province.shape[0] - province[province['long']==0].shape[0]

12626

In [18]:
# same rows with lat=0 and long=0, let's exclude them
province = province.drop(province[province['lat']==0].index)

In [19]:
# let's verify the shape of the clean dataset
province.shape

(12626, 12)

## First look to 'Regions' Dataset

In [20]:
region.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2478 entries, 0 to 2477
Data columns (total 20 columns):
casi_testati                  1323 non-null float64
codice_regione                2478 non-null int64
data                          2478 non-null object
deceduti                      2478 non-null int64
denominazione_regione         2478 non-null object
dimessi_guariti               2478 non-null int64
isolamento_domiciliare        2478 non-null int64
lat                           2478 non-null float64
long                          2478 non-null float64
note_en                       2478 non-null object
note_it                       2478 non-null object
nuovi_positivi                2478 non-null int64
ricoverati_con_sintomi        2478 non-null int64
stato                         2478 non-null object
tamponi                       2478 non-null int64
terapia_intensiva             2478 non-null int64
totale_casi                   2478 non-null int64
totale_ospedalizzati          24

We can see that casi_testati has many null values: indeed these data are available since 19-04-2020 

In [78]:
region.casi_testati.fillna(0)
region.casi_testati


0            NaN
1            NaN
2            NaN
3            NaN
4            NaN
5            NaN
6            NaN
7            NaN
8            NaN
9            NaN
10           NaN
11           NaN
12           NaN
13           NaN
14           NaN
15           NaN
16           NaN
17           NaN
18           NaN
19           NaN
20           NaN
21           NaN
22           NaN
23           NaN
24           NaN
25           NaN
26           NaN
27           NaN
28           NaN
29           NaN
          ...   
2448    243025.0
2449    104225.0
2450     62546.0
2451    156719.0
2452    214853.0
2453     57050.0
2454     60357.0
2455     13182.0
2456    384389.0
2457     65606.0
2458     36185.0
2459     38276.0
2460     85241.0
2461    126398.0
2462    266062.0
2463     99462.0
2464    257265.0
2465     73012.0
2466    565933.0
2467     76777.0
2468     19445.0
2469    244951.0
2470    105780.0
2471     63646.0
2472    157946.0
2473    215914.0
2474     57678.0
2475     60857

In [21]:
region.columns

Index(['casi_testati', 'codice_regione', 'data', 'deceduti',
       'denominazione_regione', 'dimessi_guariti', 'isolamento_domiciliare',
       'lat', 'long', 'note_en', 'note_it', 'nuovi_positivi',
       'ricoverati_con_sintomi', 'stato', 'tamponi', 'terapia_intensiva',
       'totale_casi', 'totale_ospedalizzati', 'totale_positivi',
       'variazione_totale_positivi'],
      dtype='object')

In [22]:
region.describe(include='all')

,casi_testati,codice_regione,data,deceduti,denominazione_regione,dimessi_guariti,isolamento_domiciliare,lat,long,note_en,note_it,nuovi_positivi,ricoverati_con_sintomi,stato,tamponi,terapia_intensiva,totale_casi,totale_ospedalizzati,totale_positivi,variazione_totale_positivi
count,1323.000000,2478.000000,2478,2478.000000,2478,2478.000000,2478.000000,2478.000000,2478.000000,2478,2478,2478.000000,2478.000000,2478,2478.000000,2478.000000,2478.000000,2478.000000,2478.00000,2478.000000
unique,NaN,NaN,118,NaN,21,NaN,NaN,NaN,NaN,49,49,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,2020-04-20T17:00:00,NaN,Sardegna,NaN,NaN,NaN,NaN,,,NaN,NaN,ITA,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,21,NaN,118,NaN,NaN,NaN,NaN,2430,2430,NaN,NaN,2478,NaN,NaN,NaN,NaN,NaN,NaN
mean,96127.302343,11.857143,NaN,975.100081,NaN,3445.033495,2031.019370,43.046293,12.225955,NaN,NaN,96.315174,648.023002,NaN,89287.742131,74.105327,7173.281275,722.128329,2753.14770,8.470944
std,98890.620593,6.275065,NaN,2615.799905,NaN,7969.149307,4136.511892,2.489283,2.659484,NaN,NaN,240.287082,1586.464835,NaN,145630.765996,167.399890,14929.226778,1747.580746,5608.91852,224.406621
min,3482.000000,1.000000,NaN,0.000000,NaN,0.000000,0.000000,38.115697,7.320149,NaN,NaN,-229.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.00000,-5339.000000
25%,31178.000000,7.000000,NaN,24.000000,NaN,33.000000,97.000000,41.125596,11.121231,NaN,NaN,2.000000,26.000000,NaN,5626.750000,2.000000,436.000000,32.000000,142.25000,-26.000000
50%,58383.000000,12.000000,NaN,194.500000,NaN,745.000000,601.500000,43.616760,12.388247,NaN,NaN,15.000000,122.000000,NaN,34797.500000,17.000000,2450.000000,140.000000,786.00000,0.000000
75%,127865.000000,17.000000,NaN,565.000000,NaN,2620.000000,1882.000000,45.434905,13.768136,NaN,NaN,79.000000,518.750000,NaN,102077.500000,69.750000,6162.750000,585.500000,2612.25000,30.000000


The dataset of Regions contains 2415 rows and 20 variables each. Every observation is an update of some numbers about the spread of Coronavirus in Italy. 
For 115 days and for all italian regions we have:
- number of hospitalized with sympthoms (on average 661 per day, with minimum 0 and maximum of about 12 100);
- number of people in intensive care unit (on average 75 per day, with minimum 0 and maximum of about 1 400);
- total number of hospitalized (on average 737 per day, with minimum 0 and maximum 13 400);
- number of people house-isolated (on average 2 100 per day, with minimum 0 and maximum of about 30 400);
- number of tested positive  (on average 2 800 per day, with minimum 0 and maximum of about 37 300);
- variation of new tested positive (on average 9 per day, with minimum 0 and maximum of about 1 950);
- number of new tested positive (on average 98 per day, with minimum 0 and maximum of about 3 250);
- number discharged healed (on average 3 300 per day, with a minimum of 0 and a maximum of about 60 850);
- number of deaths (on average 950 per day, with a minimum of 0 and a maximum of about 16 500);
- number of total cases (on average 7 100 per day, with a minimum of 0 and a mamimum of about 92 300);
- number of swabs (on average 85 500 per day, with a minimum of 0 and a maximum of about 917 900);
- number of tested cases (on average 93 800 per day, with a minimum of 3 500 and a maximum of about 547 100).

This dataset can be integrated with the province dataset to integrate the information about the number of cases within the cities of a region.

## Some Graphical Representations

First it is necessary to trasform the date in a 'datetime' object.

In [23]:
display(region['data'][0], province['data'][0])   # check the date format

'2020-02-24T18:00:00'

'2020-02-24T18:00:00'

In [24]:
# add new date as datetime object
region['date'] = [datetime.strptime(x, "%Y-%m-%dT%H:%M:%S").date() for x in region['data']] 
province['date'] = [datetime.strptime(x, "%Y-%m-%dT%H:%M:%S").date() for x in province['data']] 

In [25]:
region['date']

0       2020-02-24
1       2020-02-24
2       2020-02-24
3       2020-02-24
4       2020-02-24
5       2020-02-24
6       2020-02-24
7       2020-02-24
8       2020-02-24
9       2020-02-24
10      2020-02-24
11      2020-02-24
12      2020-02-24
13      2020-02-24
14      2020-02-24
15      2020-02-24
16      2020-02-24
17      2020-02-24
18      2020-02-24
19      2020-02-24
20      2020-02-24
21      2020-02-25
22      2020-02-25
23      2020-02-25
24      2020-02-25
25      2020-02-25
26      2020-02-25
27      2020-02-25
28      2020-02-25
29      2020-02-25
           ...    
2448    2020-06-19
2449    2020-06-19
2450    2020-06-19
2451    2020-06-19
2452    2020-06-19
2453    2020-06-19
2454    2020-06-19
2455    2020-06-19
2456    2020-06-19
2457    2020-06-20
2458    2020-06-20
2459    2020-06-20
2460    2020-06-20
2461    2020-06-20
2462    2020-06-20
2463    2020-06-20
2464    2020-06-20
2465    2020-06-20
2466    2020-06-20
2467    2020-06-20
2468    2020-06-20
2469    2020

### Number of total cases and deaths during the time.
 No distinctions between regions or cities.

In [26]:
region.plot(y=['totale_casi', 'deceduti'], x='date', rot=45);  

From the plot we see that the number of people infected by the Covid-19 has increased very rapidly in the first month (from the 1st of March to the first of April) and that it has continued increasing quite rapidely. The number of deaths has increased in the fist two months but now it seems quite constant.

### Time series of the counts provided
Distinction by regions

In [27]:
regions_names = region['denominazione_regione'].unique() 

# define the figure size and grid layout properties
figsize = (20, 40)
cols = 2
rows = len(regions_names) // cols + 1

def trim_axs(axs, N):
    """ Reduce *axs* to *N* Axes. 
    All further Axes are removed from the figure.
    """
    axs = axs.flat
    for ax in axs[N:]:
        ax.remove()
    return axs[:N]

In [28]:
fig1, axs = plt.subplots(rows, cols, figsize=figsize, constrained_layout=True)
axs = trim_axs(axs, len(regions_names))
for ax, reg in zip(axs, regions_names):
    ax.set_title(str(reg))
    df_r = region[region['denominazione_regione']==reg]
    ax.plot(df_r['date'], df_r['ricoverati_con_sintomi'], label='synthoms')
    ax.plot(df_r['date'], df_r['terapia_intensiva'], label='intensive care')     
    ax.plot(df_r['date'], df_r['totale_ospedalizzati'], label='hospitalized')   
    ax.plot(df_r['date'], df_r['isolamento_domiciliare'], label='isolating')
    ax.plot(df_r['date'], df_r['totale_positivi'], label='positive')
    ax.plot(df_r['date'], df_r['nuovi_positivi'], label='new positive')
    ax.plot(df_r['date'], df_r['dimessi_guariti'], label='healed')
    ax.plot(df_r['date'], df_r['deceduti'], label='deaths')
    ax.plot(df_r['date'], df_r['totale_casi'], label='cases')    
    ax.legend()

TypeError: __init__() got an unexpected keyword argument 'constrained_layout'

Let's repeat the same plot as before fixing the ylim to analyze differences by regions.

In [29]:
fig1, axs = plt.subplots(rows, cols, figsize=figsize, constrained_layout=True)
axs = trim_axs(axs, len(regions_names))
for ax, case in zip(axs, regions_names):
    ax.set_title(str(case))
    df_r = region[region['denominazione_regione']==case]
    ax.set_ylim((0, 100000))
    ax.plot(df_r['date'], df_r['ricoverati_con_sintomi'], label='synthoms')
    ax.plot(df_r['date'], df_r['terapia_intensiva'], label='intensive care')     
    ax.plot(df_r['date'], df_r['totale_ospedalizzati'], label='hospitalized')   
    ax.plot(df_r['date'], df_r['isolamento_domiciliare'], label='isolating')
    ax.plot(df_r['date'], df_r['totale_positivi'], label='positive')
    ax.plot(df_r['date'], df_r['nuovi_positivi'], label='new positive')
    ax.plot(df_r['date'], df_r['dimessi_guariti'], label='healed')
    ax.plot(df_r['date'], df_r['deceduti'], label='deaths')
    ax.plot(df_r['date'], df_r['totale_casi'], label='cases')    
    ax.legend()

TypeError: __init__() got an unexpected keyword argument 'constrained_layout'

As we can see most of the lines of the plot can be barely seen. This is because Lombardy has very higher values for quite all the counts evaluated. 
Form the plots we can say that the regions in which we find high values (apart from Lombardy) are Veneto, Piedmont and Emilia-Romagna.

### Obtain average counts by regions

We add population data from Istat (January, 1, 2019) https://www.tuttitalia.it/province/popolazione/.

In [30]:
pop = pd.read_csv('population.csv')
pop.head()

FileNotFoundError: File b'population.csv' does not exist

In [ ]:
pop.describe()

In [ ]:
# the values are seen as strings -> let's convert them
popolazione = []
for p in pop['Popolazione']:
    popolazione.append(int(p.replace('.', ''))) # we remove the dots
pop['Popolazione'] = popolazione                # we replace the values

In [ ]:
pop.head()

In [ ]:
# the variable 'Provincia' has a different notation so it is necessary change it so that we can use it 
prov = []
for i in pop.index:
    prov.append(pop.Provincia[i][:2])

In [ ]:
# we create a dataframe with the initial of the province and its population density
df_pop_prov = pd.DataFrame({'Popolazione': pop.Popolazione, 'sigla_provincia': prov}, columns=['sigla_provincia', 'Popolazione'])
df_pop_prov

In [ ]:
province_pop = pd.merge(province, df_pop_prov, on='sigla_provincia')
province_pop

In [ ]:
# we create a dataset with the initial of regions and its population density
df_pop_reg =  pd.DataFrame(pop.groupby('Reg')['Popolazione'].sum().reset_index())
df_pop_reg

It is necessary to link the abbrevation of regions to the full name. This step is useful to add the population information to the two origina datasets.

In [ ]:
df_pop_reg.Reg.unique() 

In [ ]:
region['denominazione_regione'].unique()

In [ ]:
# we create a dictionary to link the abbrevation to the full name
reg_abbrv = {'ABR': 'Abruzzo','BAS': 'Basilicata','BOL': 'P.A. Bolzano','CAL': 'Calabria','CAM': 'Campania',
       'EMR': 'Emilia-Romagna','FVG': 'Friuli Venezia Giulia','LAZ': 'Lazio','LIG': 'Liguria',
       'LOM': 'Lombardia' ,'MAR': 'Marche','MOL': 'Molise','PIE': 'Piemonte','PUG': 'Puglia','SAR': 'Sardegna',
       'SIC':'Sicilia','TOS': 'Toscana','TAA': 'P.A. Trento','UMB': 'Umbria','VDA': "Valle d'Aosta",
       'VEN': 'Veneto'}    

In [ ]:
# we add the variable 'denominazione_regione to the dataset of the population density'
regione = []
for i in df_pop_reg.Reg:
    regione.append(reg_abbrv[i])
df_pop_reg['denominazione_regione'] = regione
df_pop_reg = df_pop_reg.drop('Reg', axis=1)

In [ ]:
region_pop = pd.merge(region, df_pop_reg, on='denominazione_regione')
region_pop

In [ ]:
df = pd.DataFrame(region_pop.groupby('denominazione_regione')['ricoverati_con_sintomi', 'terapia_intensiva',
       'totale_ospedalizzati', 'isolamento_domiciliare', 'totale_positivi',
       'variazione_totale_positivi', 'nuovi_positivi', 'dimessi_guariti',
       'deceduti', 'totale_casi', 'tamponi', 'casi_testati', 'Popolazione'].mean())
df

# Questi grafici sono brutti ma non so come 'abbellirli'

In [ ]:
labels = df.index

x = np.arange(0, 21*4, step = 4)  # the label locations

fig, ax = plt.subplots(figsize=(10,10))

ax.bar(x-2, df['ricoverati_con_sintomi']/ df['Popolazione'], label='synthoms')
ax.bar(x-1.5, df['terapia_intensiva']/ df['Popolazione'], label='intensive care')     
ax.bar(x-1, df['totale_ospedalizzati']/ df['Popolazione'], label='hospitalized')   
ax.bar(x-0.5, df['isolamento_domiciliare']/ df['Popolazione'], label='isolating')
ax.bar(x, df['totale_positivi']/ df['Popolazione'], label='positive')
ax.bar(x+0.5, df['nuovi_positivi']/ df['Popolazione'], label='new positive')
ax.bar(x+1, df['dimessi_guariti']/ df['Popolazione'], label='healed')
ax.bar(x+1.5, df['deceduti']/ df['Popolazione'], label='deaths')
ax.bar(x+2, df['totale_casi']/ df['Popolazione'], label='cases')  

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_title('Proportions respect to the actual population density by region ')
ax.set_xticks(x)
ax.set_xticklabels(labels, rotation=90)
ax.legend()

fig.tight_layout()
plt.show();

In [ ]:
df.drop('Popolazione', axis=1).plot.bar(figsize=(15,10));

In [ ]:
df.plot.box(figsize=(15,10), rot=90);

### Italian scatterplot of cases
Let's use the latitude and longitude

In [ ]:
# be sure that longitude and latitude do not change 
df_1 = region.groupby('denominazione_regione')['lat', 'long']
df_1.head()

In [ ]:
# build an aggregated dataset in which we consider also the latitude and longitude
d = pd.DataFrame(region.groupby('denominazione_regione')['ricoverati_con_sintomi', 'terapia_intensiva',
       'totale_ospedalizzati', 'isolamento_domiciliare', 'totale_positivi',
       'variazione_totale_positivi', 'nuovi_positivi', 'dimessi_guariti',
       'deceduti', 'totale_casi', 'tamponi', 'casi_testati', 'lat', 'long'].mean())


In [ ]:
d['lat'] # see if it works

In [ ]:
# total cases by position
lat = d['lat']
long = d['long']
area = np.pi * (d['totale_casi']/500)**2 # dimension of the point

fig = plt.figure(figsize=(12,7))
plt.scatter(long, lat, s=area, alpha=0.5)
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.title('Number of cases for each region');

In the figure we exploit the position of each region (with latitude and longitude) to build a sort of Italy-shaped scatterplot. The dimension of each point is proportional to the number of cases registred in each region.

### Pie-plot of the contribution of every region in the main tallies

In [ ]:
# new df including only some tally
dd = pd.DataFrame(region.groupby('denominazione_regione')['ricoverati_con_sintomi', 'terapia_intensiva',
       'isolamento_domiciliare', 'totale_positivi',
       'dimessi_guariti','deceduti', 'totale_casi', 'tamponi'].mean())


In [ ]:
sns.palplot(sns.color_palette("tab20b"), 21)

In [ ]:
dd.plot.pie(subplots=True, figsize=(100,100), legend=False, colormap = "tab20");
#click on the image to zoom

In every pie plot comes out that Lombardy is the italian region that the most has benn dameged by Covid-19.
The majority of hospitalized, deaths and positive cases have been registred in Lombardy with a huge difference from the other regions. 

Instead, the number of swaps in Lombardy is not as highly different as before. Veneto seem to have the same number of swaps as the Lombardy.

## Merging the two data sets
In order to make the analysis more interesting, we can merge the two datasets (regions and province) to build one hierarchical DataFrame.


#### Some tries: try to merge a subset of the two dataset

In [31]:
r = region[:5]
p = province[:5]
display(r, p)

,casi_testati,codice_regione,data,deceduti,denominazione_regione,dimessi_guariti,isolamento_domiciliare,lat,long,note_en,...,nuovi_positivi,ricoverati_con_sintomi,stato,tamponi,terapia_intensiva,totale_casi,totale_ospedalizzati,totale_positivi,variazione_totale_positivi,date
0,NaN,13,2020-02-24T18:00:00,0,Abruzzo,0,0,42.351222,13.398438,,...,0,0,ITA,5,0,0,0,0,0,2020-02-24
1,NaN,17,2020-02-24T18:00:00,0,Basilicata,0,0,40.639471,15.805148,,...,0,0,ITA,0,0,0,0,0,0,2020-02-24
2,NaN,21,2020-02-24T18:00:00,0,P.A. Bolzano,0,0,46.499335,11.356624,,...,0,0,ITA,1,0,0,0,0,0,2020-02-24
3,NaN,18,2020-02-24T18:00:00,0,Calabria,0,0,38.905976,16.594402,,...,0,0,ITA,1,0,0,0,0,0,2020-02-24
4,NaN,15,2020-02-24T18:00:00,0,Campania,0,0,40.839566,14.250850,,...,0,0,ITA,10,0,0,0,0,0,2020-02-24


,codice_provincia,codice_regione,data,denominazione_provincia,denominazione_regione,lat,long,note_en,note_it,sigla_provincia,stato,totale_casi,date
0,69,13,2020-02-24T18:00:00,Chieti,Abruzzo,42.351032,14.167546,,,CH,ITA,0,2020-02-24
1,66,13,2020-02-24T18:00:00,L'Aquila,Abruzzo,42.351222,13.398438,,,AQ,ITA,0,2020-02-24
2,68,13,2020-02-24T18:00:00,Pescara,Abruzzo,42.464584,14.213648,,,PE,ITA,0,2020-02-24
3,67,13,2020-02-24T18:00:00,Teramo,Abruzzo,42.658918,13.704400,,,TE,ITA,0,2020-02-24
5,77,17,2020-02-24T18:00:00,Matera,Basilicata,40.667512,16.597924,,,MT,ITA,0,2020-02-24


In [32]:
t = pd.merge(r, p)
t

,casi_testati,codice_regione,data,deceduti,denominazione_regione,dimessi_guariti,isolamento_domiciliare,lat,long,note_en,...,tamponi,terapia_intensiva,totale_casi,totale_ospedalizzati,totale_positivi,variazione_totale_positivi,date,codice_provincia,denominazione_provincia,sigla_provincia
0,NaN,13,2020-02-24T18:00:00,0,Abruzzo,0,0,42.351222,13.398438,,...,5,0,0,0,0,0,2020-02-24,66,L'Aquila,AQ


In [33]:
t = pd.merge(r, p, how='outer')
t

,casi_testati,codice_regione,data,deceduti,denominazione_regione,dimessi_guariti,isolamento_domiciliare,lat,long,note_en,...,tamponi,terapia_intensiva,totale_casi,totale_ospedalizzati,totale_positivi,variazione_totale_positivi,date,codice_provincia,denominazione_provincia,sigla_provincia
0,NaN,13,2020-02-24T18:00:00,0.0,Abruzzo,0.0,0.0,42.351222,13.398438,,...,5.0,0.0,0,0.0,0.0,0.0,2020-02-24,66.0,L'Aquila,AQ
1,NaN,17,2020-02-24T18:00:00,0.0,Basilicata,0.0,0.0,40.639471,15.805148,,...,0.0,0.0,0,0.0,0.0,0.0,2020-02-24,NaN,NaN,NaN
2,NaN,21,2020-02-24T18:00:00,0.0,P.A. Bolzano,0.0,0.0,46.499335,11.356624,,...,1.0,0.0,0,0.0,0.0,0.0,2020-02-24,NaN,NaN,NaN
3,NaN,18,2020-02-24T18:00:00,0.0,Calabria,0.0,0.0,38.905976,16.594402,,...,1.0,0.0,0,0.0,0.0,0.0,2020-02-24,NaN,NaN,NaN
4,NaN,15,2020-02-24T18:00:00,0.0,Campania,0.0,0.0,40.839566,14.250850,,...,10.0,0.0,0,0.0,0.0,0.0,2020-02-24,NaN,NaN,NaN
5,NaN,13,2020-02-24T18:00:00,NaN,Abruzzo,NaN,NaN,42.351032,14.167546,,...,NaN,NaN,0,NaN,NaN,NaN,2020-02-24,69.0,Chieti,CH
6,NaN,13,2020-02-24T18:00:00,NaN,Abruzzo,NaN,NaN,42.464584,14.213648,,...,NaN,NaN,0,NaN,NaN,NaN,2020-02-24,68.0,Pescara,PE
7,NaN,13,2020-02-24T18:00:00,NaN,Abruzzo,NaN,NaN,42.658918,13.704400,,...,NaN,NaN,0,NaN,NaN,NaN,2020-02-24,67.0,Teramo,TE
8,NaN,17,2020-02-24T18:00:00,NaN,Basilicata,NaN,NaN,40.667512,16.597924,,...,NaN,NaN,0,NaN,NaN,NaN,2020-02-24,77.0,Matera,MT


In [34]:
t = pd.merge(r, p, )
t

,casi_testati,codice_regione,data,deceduti,denominazione_regione,dimessi_guariti,isolamento_domiciliare,lat,long,note_en,...,tamponi,terapia_intensiva,totale_casi,totale_ospedalizzati,totale_positivi,variazione_totale_positivi,date,codice_provincia,denominazione_provincia,sigla_provincia
0,NaN,13,2020-02-24T18:00:00,0,Abruzzo,0,0,42.351222,13.398438,,...,5,0,0,0,0,0,2020-02-24,66,L'Aquila,AQ


### True dataset

In [36]:
province.loc[province.denominazione_regione=='Toscana', 'sigla_provincia'].unique()

array(['AR', 'FI', 'GR', 'LI', 'LU', 'MS', 'PI', 'PT', 'PO', 'SI'], dtype=object)

In [37]:
toscana = province.loc[province.denominazione_regione=='Toscana']
toscana.loc[province.sigla_provincia=='AR', 'date'].unique()

array([datetime.date(2020, 2, 24), datetime.date(2020, 2, 25),
       datetime.date(2020, 2, 26), datetime.date(2020, 2, 27),
       datetime.date(2020, 2, 28), datetime.date(2020, 2, 29),
       datetime.date(2020, 3, 1), datetime.date(2020, 3, 2),
       datetime.date(2020, 3, 3), datetime.date(2020, 3, 4),
       datetime.date(2020, 3, 5), datetime.date(2020, 3, 6),
       datetime.date(2020, 3, 7), datetime.date(2020, 3, 8),
       datetime.date(2020, 3, 9), datetime.date(2020, 3, 10),
       datetime.date(2020, 3, 11), datetime.date(2020, 3, 12),
       datetime.date(2020, 3, 13), datetime.date(2020, 3, 14),
       datetime.date(2020, 3, 15), datetime.date(2020, 3, 16),
       datetime.date(2020, 3, 17), datetime.date(2020, 3, 18),
       datetime.date(2020, 3, 19), datetime.date(2020, 3, 20),
       datetime.date(2020, 3, 21), datetime.date(2020, 3, 22),
       datetime.date(2020, 3, 23), datetime.date(2020, 3, 24),
       datetime.date(2020, 3, 25), datetime.date(2020, 3, 26),
 

In [38]:
index = []
for r in province.denominazione_regione.unique():
    reg = province.loc[province.denominazione_regione==r] 
    for p in reg['sigla_provincia'].unique():
        prov = reg.loc[reg.sigla_provincia==p]
        for dta in prov['date'].unique():
            index.append((r, p, dta))
index

[('Abruzzo', 'CH', datetime.date(2020, 2, 24)),
 ('Abruzzo', 'CH', datetime.date(2020, 2, 25)),
 ('Abruzzo', 'CH', datetime.date(2020, 2, 26)),
 ('Abruzzo', 'CH', datetime.date(2020, 2, 27)),
 ('Abruzzo', 'CH', datetime.date(2020, 2, 28)),
 ('Abruzzo', 'CH', datetime.date(2020, 2, 29)),
 ('Abruzzo', 'CH', datetime.date(2020, 3, 1)),
 ('Abruzzo', 'CH', datetime.date(2020, 3, 2)),
 ('Abruzzo', 'CH', datetime.date(2020, 3, 3)),
 ('Abruzzo', 'CH', datetime.date(2020, 3, 4)),
 ('Abruzzo', 'CH', datetime.date(2020, 3, 5)),
 ('Abruzzo', 'CH', datetime.date(2020, 3, 6)),
 ('Abruzzo', 'CH', datetime.date(2020, 3, 7)),
 ('Abruzzo', 'CH', datetime.date(2020, 3, 8)),
 ('Abruzzo', 'CH', datetime.date(2020, 3, 9)),
 ('Abruzzo', 'CH', datetime.date(2020, 3, 10)),
 ('Abruzzo', 'CH', datetime.date(2020, 3, 11)),
 ('Abruzzo', 'CH', datetime.date(2020, 3, 12)),
 ('Abruzzo', 'CH', datetime.date(2020, 3, 13)),
 ('Abruzzo', 'CH', datetime.date(2020, 3, 14)),
 ('Abruzzo', 'CH', datetime.date(2020, 3, 15)),
 

In [39]:
indexes = pd.MultiIndex.from_tuples(index)

In [40]:
new_multi_data = pd.DataFrame(total, index=indexes)


NameError: name 'total' is not defined

In [ ]:
new_multi_data.head()

In [ ]:
total.reindex(indexes)

In [ ]:
vars_to_add = province[]

In [ ]:
province.columns

In [ ]:
total[total['denominazione_regione']=='Toscana'].head(15)

In [ ]:
region_i = region.set_index('denominazione_regione')
province_i = province.set_index(['denominazione_regione', 'sigla_provincia'])
display(region_i, province_i)

In [ ]:
total = pd.merge(region, province, left_on='denominazione_regione', right_on='denominazione_regione')
total
total.set_index(['denominazione_regione', 'sigla_provincia', 'date_x'])

In [ ]:
total.loc[total['denominazione_regione']=='Abruzzo'].set_index(['sigla_provincia', 'date_x'])

In [41]:
region.shape


(2478, 21)

It is possible to merge the two datasets according the variables 'data' and 'denominazione_regione', which are present in both the DataFrames.
It is important to change the data format too, beacause some data record differs in the hour although they refer to the same day:

In [59]:
sum(region.data.unique() != province.data.unique())

3

In [63]:
region['data'] = pd.to_datetime(region['data']).dt.date
province['data'] = pd.to_datetime(province['data']).dt.date
sum(region.data.unique() != province.data.unique())

0

In [64]:
cols_prov = ['codice_provincia', 'data', 'denominazione_provincia', 'denominazione_regione', 'lat', 'long', 'sigla_provincia']
merged = pd.merge(province[cols_prov], region, on = ['denominazione_regione', 'data'], suffixes = ["_prov", "_reg"])

In [71]:
merged.iloc[:10]

codice_provincia  \
denominazione_regione sigla_provincia data                           
Abruzzo               AQ              2020-02-24                66   
                                      2020-02-25                66   
                                      2020-02-26                66   
                                      2020-02-27                66   
                                      2020-02-28                66   
                                      2020-02-29                66   
                                      2020-03-01                66   
                                      2020-03-02                66   
                                      2020-03-03                66   
                                      2020-03-04                66   

                                                 denominazione_provincia  \
denominazione_regione sigla_provincia data                                 
Abruzzo               AQ              2020-02-24                L'Aquila   
                                      2020-02-25                L'Aquila   
                                      2020-02-26                L'Aquila   
                                      2020-02-27                L'Aquila   
                                      2020-02-28                L'Aquila   
                                      2020-02-29                L'Aquila   
                                      2020-03-01                L'Aquila   
                                      2020-03-02                L'Aquila   
                                      2020-03-03                L'Aquila   
                                      2020-03-04                L'Aquila   

                                                   lat_prov  long_prov  \
denominazione_regione sigla_provincia data                               
Abruzzo               AQ              2020-02-24  42.351222  13.398438   
                                      2020-02-25  42.351222  13.398438   
                                      2020-02-26  42.351222  13.398438   
                                      2020-02-27  42.351222  13.398438   
                                      2020-02-28  42.351222  13.398438   
                                      2020-02-29  42.351222  13.398438   
                                      2020-03-01  42.351222  13.398438   
                                      2020-03-02  42.351222  13.398438   
                                      2020-03-03  42.351222  13.398438   
                                      2020-03-04  42.351222  13.398438   

                                                  casi_testati  \
denominazione_regione sigla_provincia data                       
Abruzzo               AQ              2020-02-24           NaN   
                                      2020-02-25           NaN   
                                      2020-02-26           NaN   
                                      2020-02-27           NaN   
                                      2020-02-28           NaN   
                                      2020-02-29           NaN   
                                      2020-03-01           NaN   
                                      2020-03-02           NaN   
                                      2020-03-03           NaN   
                                      2020-03-04           NaN   

                                                  codice_regione  deceduti  \
denominazione_regione sigla_provincia data                                   
Abruzzo               AQ              2020-02-24              13         0   
                                      2020-02-25              13         0   
                                      2020-02-26              13         0   
                                      2020-02-27              13         0   
                                      2020-02-28              13         0   
                                      2020-02-29              13    

In [65]:
region.shape

(2478, 21)

In [66]:
province.shape

(12626, 13)

In [67]:
merged.shape

(12626, 26)

Row number = max(region, province) 12626;  
column number = 7(selected by province) + 21 - 2(mergedby) = 26

In [69]:
merged.set_index(['denominazione_regione','sigla_provincia', 'data'], inplace = True) #faccio index
merged.sort_index(inplace=True)


In [70]:
merged.iloc[:10]

codice_provincia  \
denominazione_regione sigla_provincia data                           
Abruzzo               AQ              2020-02-24                66   
                                      2020-02-25                66   
                                      2020-02-26                66   
                                      2020-02-27                66   
                                      2020-02-28                66   
                                      2020-02-29                66   
                                      2020-03-01                66   
                                      2020-03-02                66   
                                      2020-03-03                66   
                                      2020-03-04                66   

                                                 denominazione_provincia  \
denominazione_regione sigla_provincia data                                 
Abruzzo               AQ              2020-02-24                L'Aquila   
                                      2020-02-25                L'Aquila   
                                      2020-02-26                L'Aquila   
                                      2020-02-27                L'Aquila   
                                      2020-02-28                L'Aquila   
                                      2020-02-29                L'Aquila   
                                      2020-03-01                L'Aquila   
                                      2020-03-02                L'Aquila   
                                      2020-03-03                L'Aquila   
                                      2020-03-04                L'Aquila   

                                                   lat_prov  long_prov  \
denominazione_regione sigla_provincia data                               
Abruzzo               AQ              2020-02-24  42.351222  13.398438   
                                      2020-02-25  42.351222  13.398438   
                                      2020-02-26  42.351222  13.398438   
                                      2020-02-27  42.351222  13.398438   
                                      2020-02-28  42.351222  13.398438   
                                      2020-02-29  42.351222  13.398438   
                                      2020-03-01  42.351222  13.398438   
                                      2020-03-02  42.351222  13.398438   
                                      2020-03-03  42.351222  13.398438   
                                      2020-03-04  42.351222  13.398438   

                                                  casi_testati  \
denominazione_regione sigla_provincia data                       
Abruzzo               AQ              2020-02-24           NaN   
                                      2020-02-25           NaN   
                                      2020-02-26           NaN   
                                      2020-02-27           NaN   
                                      2020-02-28           NaN   
                                      2020-02-29           NaN   
                                      2020-03-01           NaN   
                                      2020-03-02           NaN   
                                      2020-03-03           NaN   
                                      2020-03-04           NaN   

                                                  codice_regione  deceduti  \
denominazione_regione sigla_provincia data                                   
Abruzzo               AQ              2020-02-24              13         0   
                                      2020-02-25              13         0   
                                      2020-02-26              13         0   
                                      2020-02-27              13         0   
                                      2020-02-28              13         0   
                                      2020-02-29              13    